# YOLO Architecture

![](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*YAXKzGS79bL8vtTt52km_g.png)



- Input size: (448, 448, 3)
- Conv layer(`Dartnet`)
    - 6층의 Conv block으로 구성
    - 1, 2, 6번 Conv block은 `Conv2d` + `BatchNorm` + `LeakyReLU(0.1)` + `MaxPool2d`로 구성
    - 나머지 Conv block은 여러 개의 `Conv2d` + `BatchNorm` + `LeakyReLU(0.1)`로 구성되며 Bottleneck 구조가 포함
- FCL(Fully Connected Layer)
    - Feature map -> Flattened vector
    - Flattened vector -> Output
- Output size: (S, S, B * 5 + C)
    - S: 격자의 수(논문 기준 7)
    - B: 격자 당 바운딩 박스 수
    - C: 클래스 수
    - * 5: (cx, cy, w, h, confidence)
![](https://miro.medium.com/v2/resize:fit:1276/format:webp/1*52e_9kGctvspNvTy949pfA.jpeg)

In [ ]:
class ConvBlock(nn.Module):
	def __init__(self):
		super(ConvBlock, self).__init__()
		


In [ ]:
class YOLOv1(nn.Module):
	def __init__(self, S=7, B=2, C=20):
		self.S = S
		self.B = B
		self.C = C

		self.darknet = nn.Sequential(
			ConvBlock(3, 64, kernel_size=7, stride=2, padding=3),
			nn.MaxPool2d(2, 2),
			
		)


In [1]:
import torch
import torch.nn as nn

# padding = kernel_size - stride / 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class YOLOv1(nn.Module):
	def __init__(self, S=7, B=2, C=20):
		super(YOLOv1, self).__init__()
		self.S = S  # 그리드 셀 개수 (7x7)
		self.B = B  # 각 그리드에서 예측하는 바운딩 박스 개수 (2개)
		self.C = C  # 클래스 개수 (예: 20개의 클래스)
		self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2 ,padding=4)
		self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
		self.lkrelu = nn.LeakyReLU(0.1)
		self.backbone = [
			[('c', 7, 64, 2), 'p'], 
			[('c', 3, 192, 1), 'p', ('c', 1, 128, 1), ('c', 3, 256), ('c', 1, 512), ('c', 3, 512), 'p'],
			[[('c', 1, 256, 1), ('c', 3, 512, 1), 4], ('c', 1, 512, 1), ('c', 3, 1024, 1), 'p'],
			[[('c', 1, 512, 1), ('c', 3, 1024, 1), 2], ('c', 3, 1024, 1), ('c', 3, 1024, 2)],
			[[('c', 3, 1024), 2]]
		]

	def forward(self, x):
		
		for layer in self.backbone:
			for i in range(len(layer)):
				if layer[i] == 'p':
					x = self.pool(x)
				else:
					l = nn.Sequential(
						
					)



SyntaxError: incomplete input (2991979975.py, line 26)

In [5]:
import torch.optim as optim

# 모델과 손실 함수 정의
model = YOLOv1()
model = model.to(device)

In [11]:
dummy = torch.rand(1, 3, 448, 448).to(device)
dummy_pred = model(dummy)
print(dummy_pred.shape)

torch.Size([1, 7, 7, 30])


# LOSS

![](https://miro.medium.com/v2/resize:fit:1018/format:webp/1*smK6Jqarqw09nA7vnQexAg.png)

In [9]:
class YoloLoss(nn.Module):
    def __init__(self, S=7, B=2, C=20):
        super(YoloLoss, self).__init__()
        self.S = S  # 그리드 개수 (7x7)
        self.B = B  # 바운딩 박스 수 (각 그리드에서 2개의 박스 예측)
        self.C = C  # 클래스 개수 (예: 20개의 클래스)

        self.mse = nn.MSELoss(reduction='sum')  # MSE 손실을 사용
        self.lambda_coord = 5  # 좌표 손실 가중치
        self.lambda_noobj = 0.5  # 객체가 없는 그리드에 대한 손실 가중치

    def forward(self, predictions, target):
        """
        predictions: [batch_size, S, S, (C + B*5)]
        target: [batch_size, S, S, (C + B*5)]
        """

        # 객체가 있는 셀을 위한 마스크
        obj_mask = target[..., 4] > 0  # 객체가 있는 경우 마스크
        no_obj_mask = target[..., 4] == 0  # 객체가 없는 경우 마스크

        # 객체가 있는 셀의 손실 계산
        # 좌표 손실 (Localization Loss)
        box_predictions = predictions[..., :4]
        box_target = target[..., :4]
        coord_loss = self.lambda_coord * self.mse(box_predictions[obj_mask], box_target[obj_mask])

        # Confidence 손실 (객체가 있는 경우)
        conf_predictions_obj = predictions[..., 4][obj_mask]
        conf_target_obj = target[..., 4][obj_mask]
        conf_loss_obj = self.mse(conf_predictions_obj, conf_target_obj)

        # Confidence 손실 (객체가 없는 경우)
        conf_predictions_noobj = predictions[..., 4][no_obj_mask]
        conf_target_noobj = target[..., 4][no_obj_mask]
        conf_loss_noobj = self.lambda_noobj * self.mse(conf_predictions_noobj, conf_target_noobj)

        # 클래스 손실 (Class Loss)
        class_predictions = predictions[..., 5:]
        class_target = target[..., 5:]
        class_loss = self.mse(class_predictions[obj_mask], class_target[obj_mask])

        # 전체 손실 계산
        total_loss = coord_loss + conf_loss_obj + conf_loss_noobj + class_loss
        return total_loss

In [12]:
# 손실 함수 정의
criterion = YoloLoss(S=7, B=2, C=20)

dummy_target = torch.rand(1, 7, 7, 30).to(device)

loss = criterion(dummy_pred, dummy_target)
print(loss)

tensor(759.7955, grad_fn=<AddBackward0>)
